## Introduction to BGP Analysis using Batfish

Network engineers routinely need to validate BGP configuration and session status in the network. The most common way to accomplish this is by connecting to multiple network devices and executing a series of `show ip bgp` commands. This distributed debugging is highly complex even in a moderately-sized network. Batfish makes BGP analysis extremely simple by providing an easy-to-query, centralized view of all BGP properties (process, peers, sessions) in the network. 

In this notebook, we will show how you can extract BGP configuration and session status information with Batfish.

![Analytics](https://ga-beacon.appspot.com/UA-100596389-3/open-source/pybatfish/jupyter_notebooks/intro-bgp-analysis?pixel&useReferer)

In [1]:
# Import packages and load questions
%run startup.py

In [2]:
# Initialize a network and snapshot
NETWORK_NAME = "bgp_network"
SNAPSHOT_NAME = "bgp_snapshot"

SNAPSHOT_PATH = "networks/example-bgp"

bf_set_network(NETWORK_NAME)
bf_init_snapshot(SNAPSHOT_PATH, name=SNAPSHOT_NAME, overwrite=True)

'bgp_snapshot'

If you used the example we provided, the network you initialized above is illustrated below. You can download/view devices' configuration files [here](https://github.com/batfish/pybatfish/tree/master/jupyter_notebooks/networks/example-bgp).

![example-network](https://raw.githubusercontent.com/batfish/pybatfish/master/jupyter_notebooks/networks/example/example-network.png)

*** REMOVE ***now tell them about the network and the issue: Interface utilization on core routers is badly distributed

Traffic from dept disproportionately hits core2 instead of core1 *(reason: dist2 doesn't have a route to core1)*

Let's say that you are troubleshooting a problem with link utilization on your core routers. The interfaces on *as2core2* have extremely high utilization, while the ones on *as2core1* are virtually unused. The first guess would be that there is some issue with ECMP and since BGP is being used for route distribution, you would start debugging the BGP configuration

## BGP process configuration
First, let's ensure that BGP is configured on the routers in question. The `BgpProcessConfiguration` question lists BGP processes configured on a given set of nodes, or on all nodes if no parameters are given.

In [3]:
# let's check to see if BGP is configured on all of the core and distribution routers
bfq.bgpProcessConfiguration(nodes='as2core.*|as2dist.*').answer().frame()

,Node,VRF,Router_ID,Multipath_EBGP,Multipath_IBGP,Tie_Breaker,Neighbors,Route_Reflector,Multipath_Match_Mode
0,as2dist2,default,2.1.3.2,True,True,ARRIVAL_ORDER,"['2.1.2.1/32', '2.1.2.2/32', '2.34.0.0/16']",False,EXACT_PATH
1,as2core2,default,2.1.2.2,True,True,ARRIVAL_ORDER,"['2.1.1.1/32', '2.1.1.2/32', '2.1.3.1/32', '2.1.3.2/32']",True,EXACT_PATH
2,as2core1,default,2.1.2.1,True,True,ARRIVAL_ORDER,"['2.1.1.1/32', '2.1.1.2/32', '2.1.3.1/32', '2.1.3.2/32']",True,EXACT_PATH
3,as2dist1,default,2.1.3.1,True,True,ARRIVAL_ORDER,"['2.1.2.1/32', '2.1.2.2/32', '2.34.0.0/16']",False,EXACT_PATH


The results of `BgpProcessConfiguration` show that `as2core1`, `as2core2`, `as2dist1` and `as2dist2` have BGP configured. Let's look at their configured peers.

## BGP peer configuration
Next we can look for any issues with the individual BGP peers configured on `as2core1` and `as2dist2`. The `BgpPeerConfiguration` question shows all BGP peers configured on a given set of nodes, or on all nodes if no parameters are given.

In [4]:
# let's get the list of configured BGP peers on all of the core and distribution routers
bfq.bgpPeerConfiguration(nodes='as2core.*|as2dist.*').answer().frame()

,Node,VRF,Local_AS,Local_IP,Remote_AS,Remote_IP,Route_Reflector_Client,Cluster_ID,Peer_Group,Import_Policy,Export_Policy,Send_Community,Is_Passive
0,as2core1,default,2,2.1.2.1,2,2.1.3.2,True,2.1.2.1,as2,[],[],True,False
1,as2core2,default,2,2.1.2.2,2,2.1.1.2,True,2.1.2.2,as2,[],[],True,False
2,as2core2,default,2,2.1.2.2,2,2.1.3.2,True,2.1.2.2,as2,[],[],True,False
3,as2dist1,default,2,2.1.3.1,2,2.1.2.1,False,None,as2,[],[],True,False
4,as2dist2,default,2,2.1.3.2,2,2.1.2.2,False,None,as2,[],[],True,False
5,as2dist1,default,2,2.1.3.1,2,2.1.2.2,False,None,as2,[],[],True,False
6,as2core1,default,2,2.1.2.1,2,2.1.1.1,True,2.1.2.1,as2,[],[],True,False
7,as2dist2,default,2,2.1.3.2,2,2.1.2.1,False,None,as2,[],[],True,False
8,as2core1,default,2,2.1.2.1,2,2.1.3.1,True,2.1.2.1,as2,[],[],True,False
9,as2core2,default,2,2.1.2.2,2,2.1.1.1,True,2.1.2.2,as2,[],[],True,False


All of the routers have the correct number of BGP peers configured. `as2core1` and `as2core2` have 4 peers each, while `as2dist1` and `as2dist2` have 3 peers each.

## BGP session status
Next, let's see if the BGP session between the `core` routers and `dist` routers are established. The `BgpSessionStatus` question shows which compatible BGP peers establish sessions. 

In [5]:
bfq.bgpSessionStatus(nodes='as2core.*|as2dist.*', remoteNodes='as2core.*|as2dist.*').answer().frame()

,Node,VRF,Local_AS,Local_Interface,Local_IP,Remote_AS,Remote_Node,Remote_IP,Session_Type,Established_Status
0,as2core1,default,2,as2core1:Loopback0,2.1.2.1,2,as2dist1,2.1.3.1,IBGP,NOT_ESTABLISHED
1,as2core1,default,2,as2core1:Loopback0,2.1.2.1,2,as2dist2,2.1.3.2,IBGP,NOT_ESTABLISHED
2,as2core2,default,2,as2core2:Loopback0,2.1.2.2,2,as2dist1,2.1.3.1,IBGP,ESTABLISHED
3,as2core2,default,2,as2core2:Loopback0,2.1.2.2,2,as2dist2,2.1.3.2,IBGP,ESTABLISHED
4,as2dist1,default,2,as2dist1:Loopback0,2.1.3.1,2,as2core1,2.1.2.1,IBGP,NOT_ESTABLISHED
5,as2dist1,default,2,as2dist1:Loopback0,2.1.3.1,2,as2core2,2.1.2.2,IBGP,ESTABLISHED
6,as2dist2,default,2,as2dist2:Loopback0,2.1.3.2,2,as2core1,2.1.2.1,IBGP,NOT_ESTABLISHED
7,as2dist2,default,2,as2dist2:Loopback0,2.1.3.2,2,as2core2,2.1.2.2,IBGP,ESTABLISHED


Now we can see that the BGP session did *not* get established between `as2core1` and `as2dist1` or `as2dist2`, even though they seemed compatible based on the results of `BgpPeerConfiguration`.

What would be the cause for this? To find out, we can check the compatibility of the configured BGP sessions.

## BGP session compatibility
Did we miss something when we looked at the results of `BgpPeerConfiguration`? The `BgpSessionCompatibility` question shows the compatibility of BGP sessions, which should confirm whether `as2core1` and `as2dist1` or `as2dist2` are compatible. Sessions can have various compatibility statuses. A correctly configured peer will have one of the following statuses:

- `UNIQUE_MATCH`: The peer is active and has exactly one compatible remote peer.
- `DYNAMIC_MATCH`: The peer is passive and has at least one compatible remote peer.
- `UNKNOWN_REMOTE`: The peer is active, but its remote IP is not associated with a known interface. Most likely, this peer is intended to establish a session with an external device, but if not, it is improperly configured.
- `NO_MATCH_FOUND`: The peer is passive, but no active peers in the network are compatible with it. Most likely, this peer is expecting sessions to be initiated by external devices, but if not, it is improperly configured.

Other statuses may indicate basic configuration issues, such as `NO_LOCAL_IP` or `NO_REMOTE_AS`, as well as more subtle issues:

- `INVALID_LOCAL_IP`: The peer is configured with a local IP that isn't associated with a known interface.
- `HALF_OPEN`: The peer is active and its remote IP is recognized, but no compatible peer exists on the remote interface.
- `MULTIPLE_REMOTES`: The peer is active with multiple remote peers configured compatibly.

Let's drill into the configured BGP session between `as2core1` and `as2dist2`. If they are compatibly configured, we should see `UNIQUE_MATCH` for their session.

In [7]:
bfq.bgpSessionCompatibility(nodes='as2core1|as2dist2', remoteNodes='as2core1|as2dist2').answer().frame()

,Node,VRF,Local_AS,Local_Interface,Local_IP,Remote_AS,Remote_Node,Remote_IP,Session_Type,Configured_Status
0,as2core1,default,2,as2core1:Loopback0,2.1.2.1,2,as2dist2,2.1.3.2,IBGP,UNIQUE_MATCH
1,as2dist2,default,2,as2dist2:Loopback0,2.1.3.2,2,as2core1,2.1.2.1,IBGP,UNIQUE_MATCH


Looking at the results for the session between `as2core1` and `as2dist2`, we are now confident that the BGP peers are correctly configured. There must be some other problem preventing the session from being established.

While we're looking at `BgpSessionCompatibility`, let's check the rest of the network for any BGP sessions that may not be compatible. We will limit the output to peers whose configured status is *not* `UNIQUE_MATCH` or `DYNAMIC_MATCH`.

In [9]:
# we can probably skip this check here. It doesn't quite flow with the rest of the narrative
all_session_compatibility = bfq.bgpSessionCompatibility().answer().frame()
all_session_compatibility[
    (all_session_compatibility['Configured_Status'] != 'UNIQUE_MATCH')
    & (all_session_compatibility['Configured_Status'] != 'DYNAMIC_MATCH')]

,Node,VRF,Local_AS,Local_Interface,Local_IP,Remote_AS,Remote_Node,Remote_IP,Session_Type,Configured_Status
1,as1border1,default,1,None,None,666,None,3.2.2.2,EBGP_SINGLEHOP,NO_LOCAL_IP
2,as1border1,default,1,None,None,555,None,5.6.7.8,EBGP_SINGLEHOP,NO_LOCAL_IP
6,as1border2,default,1,as1border2:GigabitEthernet2/0,10.14.22.1,4,None,10.14.22.4,EBGP_SINGLEHOP,UNKNOWN_REMOTE
13,as2border2,default,2,None,None,2,None,2.1.2.2,IBGP,LOCAL_IP_UNKNOWN_STATICALLY
20,as2core2,default,2,as2core2:Loopback0,2.1.2.2,2,None,2.1.1.2,IBGP,HALF_OPEN


Of these results, the first two have no local IP configured, so will be unable to establish sessions: almost certainly an oversight that should be fixed. The third has a remote IP outside of the network Batfish is modeling, so it may be able to establish a session in the live network, but Batfish doesn't have enough information to be sure.

### TODO explain last two results

These three results do not offer an explanation for the problem at hand, so let's keep looking.

## Check routes
Now we know the BGP peers on `as2core1` and `as2dist2` are compatible, but the session is not established. Let's make sure both devices have a route to the interface of each other's BGP peers. Looking at any of the last few questions concerning these two nodes, we can see the IPs of these interfaces:

- BGP peer on `as2core1` has local IP `2.1.2.1`
- BGP peer on `as2dist2` has local IP `2.1.3.2`

To see if the devices have the necessary routes to each other, we can search the output of the `Routes` question.

In [10]:
# See if as2core1 has a route to 2.1.3.2 (the local IP for as2dist2's BGP peer)
routes = bfq.routes().answer().frame()
routes[(routes['Node'] == 'as2core1') & (routes['Network'] == '2.1.3.2/32')]

,Node,VRF,Network,Protocol,Next_Hop_IP,Next_Hop,Admin_Distance,Metric,Tag
220,as2core1,default,2.1.3.2/32,ospf,2.23.12.3,as2dist2,110,2,None


In [11]:
# See if as2dist2 has a route to 2.1.2.1 (the local IP for as2core1's BGP peer)
routes[(routes['Node'] == 'as2dist2') & (routes['Network'] == '2.1.2.1/32')]

,Node,VRF,Network,Protocol,Next_Hop_IP,Next_Hop,Admin_Distance,Metric,Tag


The mystery is solved! Node `as2dist2` has no route for its BGP peer's remote IP, so even though the BGP peer was configured correctly, the session could not be established.

Since the route for `as2dist2` Loopback interface was learnt by `as2core1` via OSPF, it would appear that we need to check the OSPF configuration on `as2core1` to ensure that it is configured to advertise it's Loopback address.

Looking at the configuration of `as2core1` we can see that the Loopback address isn't part of the OSPF process:
```
interface Loopback0
 ip address 2.1.2.1 255.255.255.255

router ospf 1
 router-id 2.1.2.1
 !network 2.0.0.0 0.255.255.255 area 1
 network 2.12.0.0 0.0.255.255 area 1
 network 2.23.0.0 0.0.255.255 area 1
 ```
 
 This explains why the BGP session wasn't established.

***
## Get involved with the Batfish community

Join our community on [Slack](https://join.slack.com/t/batfish-org/shared_invite/enQtMzA0Nzg2OTAzNzQ1LTUxOTJlY2YyNTVlNGQ3MTJkOTIwZTU2YjY3YzRjZWFiYzE4ODE5ODZiNjA4NGI5NTJhZmU2ZTllOTMwZDhjMzA) and [Github](https://github.com/batfish/batfish). 